In [2]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# CIFAR_10 is a set of 60K images 32x32 pixels on 3 channels
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32

#constant
BATCH_SIZE = 128
NB_EPOCH = 20
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert to categorical
Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
Y_test = np_utils.to_categorical(y_test, NB_CLASSES)

# float and normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
# network
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

# train
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, validation_split=VALIDATION_SPLIT, verbose=VERBOSE)
score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

#save model
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
# And the weights learned by our deep network on the training set
model.save_weights('cifar10_weights.h5', overwrite=True)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
activation_2 (Activation)    (None, 512)              

In [4]:
# deeper network
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))

# dump the network and change iterations
model.summary()
NB_EPOCH = 40

# train
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, validation_split=VALIDATION_SPLIT, verbose=VERBOSE)
score = model.evaluate(X_test, Y_test, batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

#save model
model_json = model.to_json()
open('cifar10_architecture.json', 'w').write(model_json)
# And the weights learned by our deep network on the training set
model.save_weights('cifar10_weights.h5', overwrite=True)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_4 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)       

In [6]:
# data augmentation

from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
import numpy as np
NUM_TO_AUGMENT=5

#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# augumenting
print("Augmenting training set images...")
datagen = ImageDataGenerator(rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode='nearest')
xtas, ytas = [], []
for i in range(X_train.shape[0]):
    num_aug = 0
    x = X_train[i] # (3, 32, 32)
    x = x.reshape((1,) + x.shape) # (1, 3, 32, 32)
for x_aug in datagen.flow(x, batch_size=1,
                save_to_dir='preview',
                save_prefix='cifar',
                save_format='jpeg'):
    if num_aug >= NUM_TO_AUGMENT:
        break
    xtas.append(x_aug[0])
    num_aug += 1

#fit the dataget
datagen.fit(X_train)

# dump the network and change iterations
model.summary()
NB_EPOCH = 50

# train
history = model.fit_generator(datagen.flow(X_train, Y_train,
            batch_size=BATCH_SIZE), samples_per_epoch=X_train.shape[0],
            epochs=NB_EPOCH, verbose=VERBOSE)
score = model.evaluate(X_test, Y_test,
            batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])

Augmenting training set images...
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_4 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)   

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, verbose=1, steps_per_epoch=390)`


Epoch 1/50
390/390 [==============================] - 108s 276ms/step - loss: 4.6545 - accuracy: 0.1037
Epoch 2/50
390/390 [==============================] - 108s 278ms/step - loss: 2.3065 - accuracy: 0.0994
Epoch 3/50
390/390 [==============================] - 108s 277ms/step - loss: 2.3054 - accuracy: 0.0982
Epoch 4/50
390/390 [==============================] - 107s 275ms/step - loss: 2.3068 - accuracy: 0.0978
Epoch 5/50
390/390 [==============================] - 108s 276ms/step - loss: 2.3043 - accuracy: 0.0988
Epoch 6/50
390/390 [==============================] - 107s 274ms/step - loss: 2.3075 - accuracy: 0.0967
Epoch 7/50
390/390 [==============================] - 107s 274ms/step - loss: 2.3038 - accuracy: 0.0991
Epoch 8/50
390/390 [==============================] - 108s 276ms/step - loss: 2.3041 - accuracy: 0.0988
Epoch 9/50
390/390 [==============================] - 108s 277ms/step - loss: 2.3063 - accuracy: 0.0985
Epoch 10/50
390/390 [==============================] - 107s 275m

Explain how this algorithm could result in ethical and privacy concerns if it were trained on different sets of images.

Facial recognition technology is used in all areas.  Cell phones, medical diagnosis, and even law enforcement.  Using someone's image should require informed consent.  Unfortunately, this is not always the case.  The ring doorbell, owned by Amazon, can be given to police without a user's consent.  If a person's image is used to train an algorithm, or identified through an algorithm, this could lead to other concerns.  “For example, if you use facial recognition to identify people coming into a store, should you use that identity to pull purchasing history?” (Gray, 2022).
There’s also the issue of bias within race. A 2018 study performed by MIT found that most algorithms misidentified darker-skinned women (Gray, 2022).  Data put into the algorithm needs to be as diverse as possible.  This may be difficult to achieve if the development team isn’t diverse.

References:

Gray, P. (2022, August 31). Ethical issues of Facial Recognition Technology. TechRepublic. Retrieved March 15, 2023, from https://www.techrepublic.com/article/ethical-issues-facial-recognition/ 
